# 简介

[openai-python](https://github.com/openai/openai-python)是`OpenAI`的官方`Python`库.

# 安装

```bash
pip install openai
```

# 使用

## 基本使用

首先要创建一个客户端对象.

In [3]:
import json

from openai import OpenAI

open_client = OpenAI(
    base_url="https://api.juheai.top/v1/",
    api_key='sk-RLbK5xgSbH1YsoYW8fA55c5b03864108930316F7178bF970'
)



## 文本补全

### 简单对话

In [2]:
response = open_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": f"""你好"""
        }
    ]
)
result_text = response.choices[0].message.content.strip()
print(result_text)

你好！有什么我可以帮助你的吗？


### 返回`json`格式

In [1]:
response = open_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": f"""随机生成以下json格式的数据: 
        {{
            "name": "张三",
            "age": 18,
        }}
"""
        }
    ],
    response_format={"type": "json_object"}
)
# print(response)
result_text = response.choices[0].message.content.strip()
print(result_text)

NameError: name 'open_client' is not defined

#### 返回`pydantic`模型

In [7]:
from pydantic import BaseModel
from typing import TypeVar, Type

# 定义泛型类型变量
T = TypeVar('T', bound=BaseModel)


def generate_instance_from_prompt(open_client: OpenAI, prompt: str, model: Type[T]) -> T:
    """
    调用 OpenAI API，根据提示词生成符合指定 Pydantic 类型的实例.

    :param open_client: OpenAI 客户端对象
    :param prompt: 提示词
    :param model: Pydantic 泛型类型
    :return: Pydantic 类型的实例
    """
    # 调用 OpenAI API 生成 JSON 数据
    response = open_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"""{prompt},返回的数据需要符合以下 json schema: {json.dumps(model.model_json_schema(), indent=4)}
"""
            }
        ],
        response_format={"type": "json_object"}
    )
    # print(response)
    # 提取生成的 JSON 数据
    result_text = response.choices[0].message.content.strip()

    # 解析 JSON 数据为 Pydantic 实例
    data_dict = json.loads(result_text)
    instance = model(**data_dict)
    return instance


class Person(BaseModel):
    name: str
    age: int
    
instance = generate_instance_from_prompt(open_client, "随机生成一个人的信息", Person)

print(instance)

name='李明' age=28
